In [1]:
%cd /content/drive/MyDrive/KIFIYA Projects/Rossmann-Pharmaceuticals

/content/drive/MyDrive/KIFIYA Projects/Rossmann-Pharmaceuticals


In [2]:
import sys
sys.path.append('/content/drive/MyDrive/KIFIYA Projects/Rossmann-Pharmaceuticals/scripts')
import data_processing as dp
import model_training as mt
import model_evaluation as me
import lstm_model as lm
import eda_script_1 as eda

In [3]:
train_data = eda.load_data('/content/drive/MyDrive/KIFIYA Projects/Rossmann-Pharmaceuticals/data/train.csv')
store_data = eda.load_data('/content/drive/MyDrive/KIFIYA Projects/Rossmann-Pharmaceuticals/data/store.csv')
sample_data = eda.load_data('/content/drive/MyDrive/KIFIYA Projects/Rossmann-Pharmaceuticals/data/sample_submission.csv')
test_data = eda.load_data('/content/drive/MyDrive/KIFIYA Projects/Rossmann-Pharmaceuticals/data/test.csv')

/content/drive/MyDrive/KIFIYA Projects/Rossmann-Pharmaceuticals/scripts/eda_script_1.py:16: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv(data_path)


In [4]:
data = dp.load_data('/content/drive/MyDrive/KIFIYA Projects/Rossmann-Pharmaceuticals/data/train.csv')
data = dp.extract_features(data)
data = dp.handle_missing_values(data)

/content/drive/MyDrive/KIFIYA Projects/Rossmann-Pharmaceuticals/scripts/data_processing.py:5: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file_path)
/content/drive/MyDrive/KIFIYA Projects/Rossmann-Pharmaceuticals/scripts/data_processing.py:37: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  return data.fillna(method='ffill')


In [5]:
# Check for non-numeric values in each column
import pandas as pd
feature_columns = ['Weekday', 'IsWeekend', 'DaysToHoliday', 'DaysAfterHoliday',
                       'BeginningOfMonth', 'MidMonth', 'EndOfMonth', 'Season', 'Open', 'Promo', 'StateHoliday', 'SchoolHoliday']
for col in feature_columns:
    if not pd.api.types.is_numeric_dtype(data[col]):
        print(f"Non-numeric values found in column: {col}")
        print(data[col].unique())


Non-numeric values found in column: StateHoliday
['0' 'a' 'b' 'c' 0]


In [6]:
data = dp.convert_categorical(data)

/content/drive/MyDrive/KIFIYA Projects/Rossmann-Pharmaceuticals/scripts/data_processing.py:44: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['StateHoliday'].fillna(0, inplace=True)


In [7]:
feature_columns = ['Weekday', 'IsWeekend', 'DaysToHoliday', 'DaysAfterHoliday',
                   'BeginningOfMonth', 'MidMonth', 'EndOfMonth', 'Season', 'Open', 'Promo', 'StateHoliday']
# Scale features
data = dp.scale_features(data, feature_columns)
data.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,Weekday,IsWeekend,DaysToHoliday,DaysAfterHoliday,BeginningOfMonth,MidMonth,EndOfMonth,Season,SchoolHoliday_1
0,1,5,2015-07-31,5263,555,0.452399,1.273237,-0.159217,0.501484,-0.630672,-1.734123,0.0,-0.700515,-0.700517,1.388213,0.599358,True
1,2,5,2015-07-31,6064,625,0.452399,1.273237,-0.159217,0.501484,-0.630672,-1.734123,0.0,-0.700515,-0.700517,1.388213,0.599358,True
2,3,5,2015-07-31,8314,821,0.452399,1.273237,-0.159217,0.501484,-0.630672,-1.734123,0.0,-0.700515,-0.700517,1.388213,0.599358,True
3,4,5,2015-07-31,13995,1498,0.452399,1.273237,-0.159217,0.501484,-0.630672,-1.734123,0.0,-0.700515,-0.700517,1.388213,0.599358,True
4,5,5,2015-07-31,4822,559,0.452399,1.273237,-0.159217,0.501484,-0.630672,-1.734123,0.0,-0.700515,-0.700517,1.388213,0.599358,True


# **Train-test Split**

In [8]:
# Train-test split
X_train, X_test, y_train, y_test = dp.split_data(data, feature_columns, 'Sales')

# **Train Random Forest model**

In [9]:
# Train Random Forest model
pipeline = mt.build_model_pipeline()
trained_pipeline = mt.train_model(pipeline, X_train, y_train)

# **Evaluate model**

In [10]:
mae, y_pred = me.evaluate_model(trained_pipeline, X_test, y_test)
print(f'Mean Absolute Error: {mae}')

Mean Absolute Error: 1595.1030884605473


In [11]:
# Save model
mt.save_model(trained_pipeline)

# **Feature importance**

In [23]:
# Feature importance
importance_df = me.feature_importance(trained_pipeline, feature_columns)
importance_df

,Feature,Importance
8,Open,0.770141
9,Promo,0.122803
2,DaysToHoliday,0.058261
0,Weekday,0.036730
4,BeginningOfMonth,0.003410
7,Season,0.002771
6,EndOfMonth,0.002436
5,MidMonth,0.002290
1,IsWeekend,0.000878
10,StateHoliday,0.000281


# **Time Series**

In [13]:
time_series_data = lm.prepare_time_series(data)
if lm.check_stationarity(time_series_data):
    print("The time series is stationary.")
else:
    print("The time series is not stationary.")

The time series is stationary.


# **Supervised Learning Data**

In [22]:
# Create supervised learning data
time_step = 10
X, y = lm.create_supervised_data(time_series_data['Sales'].values, time_step)
X = X.reshape(X.shape[0], X.shape[1], 1)  # Reshape for LSTM input

In [18]:
# Build and train LSTM model
lstm_model = lm.build_lstm_model((time_step, 1))
lm.train_lstm_model(lstm_model, X, y)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 5s 17ms/step - loss: 6349709.0000
Epoch 2/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 6358795.5000
Epoch 3/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 6233687.5000
Epoch 4/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 6186186.5000
Epoch 5/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 6197138.5000
Epoch 6/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 6263959.5000
Epoch 7/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 6212047.0000
Epoch 8/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 6170403.0000
Epoch 9/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 6200830.5000
Epoch 10/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 6057491.5000
Epoch 11/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 6192301.5000
Epoch 12/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 6289182.0000
Epoch 13/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 6166336.0000
Epoch 14/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms